In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=280)

from tqdm import trange
import torch
from muzero import Muzero
from environments import TicTacToe
from mcts import Tree
from network import Network
import random

my_seed = 1954

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
np.random.seed(my_seed)

M = Muzero(TicTacToe) #creating the agent
M_random = Muzero(TicTacToe) #creating the random adversary

game = TicTacToe()

nb_opti_steps = 500
nb_epochs = 15
nb_leaves_per_move = 30
nb_episodes = 15
nb_games = 100

In [5]:
np.random.seed(my_seed)

#traininig model
for _ in range(nb_epochs):
    M.REPLAY_BUFFER = []
    
    for _ in trange(nb_episodes): #playing
        with torch.no_grad():
            M.REPLAY_BUFFER.append(M.mcts.full_episode(nb_leaves_per_move))
            
    for _ in trange(nb_opti_steps): #optimisation 
        M.optimize_step()
    M.network.save('./models/ttt_candidate1.ptc') 

  0%|          | 0/500 [00:00<?, ?it/s]/home/alex/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
100%|██████████| 500/500 [00:05<00:00, 91.19it/s]


In [10]:
#Save models
np.random.seed(my_seed)

M.network.save('./models/m_201125_a.ptc')
M_random.network.save('./models/random.ptc')

In [11]:
np.random.seed(my_seed)

def play_a_game(player1, player2):
    """
    Returns the reward for player 1, since it's + if P1 wins and - if P2 wins
    """
    game = TicTacToe()
    player1.environment = game
    player2.environment = game
    
    while True: #while no one wins and it's not a draw
        
        #print("PLAYER 1: ", flush=True)
        game.play(player1.move(game.state, leaves_per_move=20)) #P1 does one move
        if game.end: #if the game is over i.e. P1 wins or it's a draw
            game.show
            return game.reward #return the reward associated to the game
        #Here the game is not yet over, it's P2's turn
        #print("PLAYER 2: ", flush=True)
        game.play(player2.move(game.state, leaves_per_move=20)) #P2 does one move
        if game.end:
            game.show #if the game is over i.e. P1 wins or it's a draw
            return -game.reward #return the reward associated to the game

In [8]:
#MuZero agent vs random
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/ttt_candidate1.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/random.ptc')

with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_a = 0
    for g in (t:=trange(nb_games)):
        score_white_a += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_a}")

with torch.no_grad():
    score_black_a = 0
    for g in (t:=trange(nb_games)):
        score_black_a -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_a}")

print(f'Final Scores = W: {score_white_a} / B: {score_black_a} over {nb_games} games')

  0%|          | 0/100 [00:00<?, ?it/s]

 ___
|xoo|
|xox|
|xxo|
 ---


Score as white: 1:   1%|          | 1/100 [00:00<00:19,  5.02it/s]

 ___
| ox|
|o x|
|oxx|
 ---


Score as white: 2:   2%|▏         | 2/100 [00:00<00:18,  5.29it/s]

...

 ___
|xox|
|xo |
|oox|
 ---


Score as black: -21:  99%|█████████▉| 99/100 [00:21<00:00,  5.19it/s]

 ___
|oxx|
|oo |
|oxx|
 ---


Score as black: -20: 100%|██████████| 100/100 [00:21<00:00,  4.69it/s]

Final Scores = W: 28 / B: -20 over 100 games


In [9]:
#random agent vs random
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/random.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/random.ptc')


with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_b = 0
    for g in (t:=trange(nb_games)):
        score_white_b += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_b}")

with torch.no_grad():
    score_black_b = 0
    for g in (t:=trange(nb_games)):
        score_black_b -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_b}")

print(f'Final Scores = W: {score_white_b} / B: {score_black_b} over {nb_games} games')

  0%|          | 0/100 [00:00<?, ?it/s]

 ___
|xoo|
|xox|
|xxo|
 ---


Score as white: 1:   1%|          | 1/100 [00:00<00:24,  3.98it/s]

 ___
| ox|
|o x|
|oxx|
 ---


Score as white: 2:   2%|▏         | 2/100 [00:00<00:22,  4.38it/s]

...

 ___
|xoo|
|oxx|
|xox|
 ---


Score as black: -35:  99%|█████████▉| 99/100 [00:20<00:00,  4.87it/s]

 ___
|o x|
|oxx|
|oxo|
 ---


Score as black: -34: 100%|██████████| 100/100 [00:20<00:00,  4.93it/s]

Final Scores = W: 34 / B: -34 over 100 games


In [12]:
#MuZ vs MuZ
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/ttt_candidate1.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/ttt_candidate1.ptc')

with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_c = 0
    for g in (t:=trange(nb_games)):
        score_white_c += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_c}")

with torch.no_grad():
    score_black_c = 0
    for g in (t:=trange(nb_games)):
        score_black_c -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_c}")

print(f'Final Scores = W: {score_white_c} / B: {score_black_c} over {nb_games} games')

  0%|          | 0/100 [00:00<?, ?it/s]

 ___
|xoo|
|xox|
|xxo|
 ---


Score as white: 1:   1%|          | 1/100 [00:00<00:21,  4.71it/s]

 ___
|xox|
|oox|
|xxo|
 ---


Score as white: 1:   2%|▏         | 2/100 [00:00<00:20,  4.71it/s]

...

 ___
|ox |
|xx |
|oxo|
 ---


Score as black: -21:  99%|█████████▉| 99/100 [00:22<00:00,  4.98it/s]

 ___
| xo|
| x |
|ox |
 ---


Score as black: -22: 100%|██████████| 100/100 [00:22<00:00,  4.44it/s]

Final Scores = W: 36 / B: -22 over 100 games


In [13]:
print(f"Score comparison over {nb_games} games \n MuZero vs Random = W: {score_white_a} & B: {score_black_a} \n Random vs Random = W: {score_white_b} & B: {score_black_b} \n MuZero vs MuZero = W: {score_white_c} & B: {score_black_c}")
#print(f"Score comparison over {nb_games} games \n MuZero vs Random = W: {score_white_a} & B: {score_black_a} \n Random vs Random = W: {score_white_b} & B: {score_black_b} \n MuZero vs MuZero = W: {score_white_c} & B: {score_black_c}")

Score comparison over 100 games 
 MuZero vs Random = W: 28 & B: -20 
 Random vs Random = W: 34 & B: -34 
 MuZero vs MuZero = W: 36 & B: -22
